In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv(
    r"C:\Users\feldm\Documents\GitHub\real-estate-price-prediction\data\raw.csv")
df.replace(to_replace="None", value=np.nan, inplace=True)
df.head()


,locality,Price,Type_property,Sale_type,Number_bedrooms,Living_area,fully_equipped_kitchen,Furnished,terrace,garden,surface_land,surface_area_plot,facades_number,Swimming_pool,building_state,fire_place
0,6800,145500,apartment,for-sale,1,40,1,0,8,NaN,40,NaN,NaN,0,Good,NaN
1,6980,185000,mixed-use-building,for-sale,4,226,1,NaN,NaN,NaN,226,100,3,0,To be done up,NaN
2,6960,228000,apartment,for-sale,4,290,0,0,NaN,300,290,NaN,4,NaN,To renovate,NaN
3,6880,249500,house,for-sale,3,176,1,0,18,620,176,620,3,0,As new,NaN
4,6870,295000,house,for-sale,6,229,1,0,NaN,NaN,229,5798,4,0,To renovate,NaN


In [9]:
# knowing how many were missing
print(df['Living_area'].isna().sum())  # 1091
print(df['facades_number'].isna().sum())  # 1091
print(df['building_state'].isna().sum())
# get rid of empty lines of information
df.dropna(subset=['Living_area', 'facades_number',
          'building_state', 'Number_bedrooms'], inplace=True)
print(df.shape)
# get rid of collumns that I don't need
df.drop("surface_area_plot", inplace=True, axis=1)
df.drop("surface_land", inplace=True, axis=1)
df.drop("Sale_type", inplace=True, axis=1)

# replace Nan into 0
df['garden'].fillna(0, inplace=True)
df['terrace'].fillna(0, inplace=True)
df['fire_place'].fillna(0, inplace=True)
df['fully_equipped_kitchen'].fillna(0, inplace=True)
df['Furnished'].fillna(0, inplace=True)
df['Swimming_pool'].fillna(0, inplace=True)

df.reset_index(inplace=True)


# modifying the types of the columns
df['Type_property'] = df['Type_property'].astype('string')
df['building_state'] = df['building_state'].astype('string')

df['garden'] = pd.to_numeric(df['garden'])
df['Number_bedrooms'] = pd.to_numeric(df['Number_bedrooms'])
df['Living_area'] = pd.to_numeric(df['Living_area'])
df['garden'] = pd.to_numeric(df['garden'])
df['facades_number'] = pd.to_numeric(df['facades_number'])
df['Swimming_pool'] = pd.to_numeric(df['Swimming_pool'])

df['terrace'] = pd.to_numeric(df['terrace'])
df['fire_place'] = pd.to_numeric(df['fire_place'])
df['fully_equipped_kitchen'] = pd.to_numeric(df['fully_equipped_kitchen'])
df['Furnished'] = pd.to_numeric(df['Furnished'])

# creating a province column
df["provinces"] = df["locality"]
df['provinces'] = np.where(df['locality'].between(
    1000, 1299), "Brussel", df['provinces'])
df['provinces'] = np.where(df['locality'].between(
    6600, 6999), "Luxembourg", df['provinces'])
df['provinces'] = np.where(df['locality'].between(
    2000, 2999), "Anvers", df['provinces'])
df['provinces'] = np.where(df['locality'].between(
    1300, 1499), "W.brabant", df['provinces'])
df['provinces'] = np.where(df['locality'].between(
    7000, 7999), "Hainaut", df['provinces'])
df['provinces'] = np.where(df['locality'].between(
    6000, 6599), "Hainaut", df['provinces'])

df['provinces'] = np.where(df['locality'].between(
    3500, 3999), "Limbourg", df['provinces'])
df['provinces'] = np.where(df['locality'].between(
    5000, 5680), "Namur", df['provinces'])
df['provinces'] = np.where(df['locality'].between(
    9000, 9999), "E.Fanders", df['provinces'])
df['provinces'] = np.where(df['locality'].between(
    1500, 1999), "F.Brabant", df['provinces'])
df['provinces'] = np.where(df['locality'].between(
    3000, 3499), "F.Brabant", df['provinces'])

df['provinces'] = np.where(df['locality'].between(
    8000, 8999), "W.Flanders", df['provinces'])
df['provinces'] = np.where(df['locality'].between(
    4000, 4999), "Liege", df['provinces'])
print(df.shape)


9964
16498
18586
(28302, 16)
(28302, 15)


In [10]:
# removing outliers from Price
p25 = df['Price'].quantile(0.25)
p75 = df['Price'].quantile(0.75)
iqr = p75-p25
upper_limit = p75 + 1.5 * iqr
lower_limit = p25 - 1.5 * iqr
df = df.drop(df[df['Price'] > upper_limit].index)
df = df.drop(df[df['Price'] < lower_limit].index)
# removing outliers from garden
p25 = df['garden'].quantile(0.25)
p75 = df['garden'].quantile(0.75)
iqr = p75-p25
upper_limit = p75 + 1.5 * iqr
lower_limit = p25 - 1.5 * iqr
df = df.drop(df[df['garden'] > upper_limit].index)
df = df.drop(df[df['garden'] < lower_limit].index)
# removing outliers from living area
p25 = df['Living_area'].quantile(0.25)
p75 = df['Living_area'].quantile(0.75)
iqr = p75-p25
upper_limit = p75 + 1.5 * iqr
lower_limit = p25 - 1.5 * iqr
df = df.drop(df[df['Living_area'] > upper_limit].index)
df = df.drop(df[df['Living_area'] < lower_limit].index)
print(df.shape)


(19950, 15)


In [11]:
# replace data by 0 and 1
df.loc[(df.terrace == 0), 'terrace'] = 0
df.loc[(df.terrace > 0), 'terrace'] = 1

df.loc[(df.fire_place == 0), 'fire_place'] = 0
df.loc[(df.fire_place > 0), 'fire_place'] = 1

df.loc[(df.garden == 0), 'garden'] = 0
df.loc[(df.garden > 0), 'garden'] = 1

# dropping false number of facade
df.drop(df[df['facades_number'] > 4].index, inplace=True)
# dropping big number of bedrooms
df.drop(df[df['Number_bedrooms'] > 100].index, inplace=True)

df.shape


(19946, 15)

In [12]:
# drop columns
df.drop("index", inplace=True, axis=1)
df.drop("locality", inplace=True, axis=1)
print(df.shape)

# getting dummies for the datasets to feed the model
df = pd.get_dummies(df, columns=['provinces'], drop_first=True)
df = pd.get_dummies(df, columns=['Type_property'], drop_first=True)
df = pd.get_dummies(df, columns=['building_state'], drop_first=True)


(19946, 13)
Price                     0
Type_property             0
Number_bedrooms           0
Living_area               0
fully_equipped_kitchen    0
Furnished                 0
terrace                   0
garden                    0
facades_number            0
Swimming_pool             0
building_state            0
fire_place                0
provinces                 0
dtype: int64


In [13]:
# saving preprocessed csv to use it for the model
df.head()
df.to_csv("cleaned_two.csv")
